<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

In [9]:
# import cell
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf 
import sklearn.linear_model
import matplotlib.pyplot as plt
import tempfile
from sklearn.utils import shuffle
import os
import datetime
# Our Model
from tensorflow import keras
from tensorflow.keras import layers
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist
from keras.layers import Dropout
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorboard.plugins.hparams import api as hp
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
# Newer Imports 
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import ReLU

In [6]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
# Your Code Starts Here - load in data and spilt
data = np.load('quickdraw10.npz')
def load_quickdraw10npz(path):
    X = path['arr_0']
    y = path['arr_1']
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.1)
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = load_quickdraw10npz(path=data)
print(f'shapes{X_train.shape,y_train.shape,X_test.shape,y_test.shape}')

shapes((80000, 784), (80000,), (10000, 784), (10000,))


## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [11]:
def final_model(neurons=64):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=784, kernel_initializer='glorot_uniform', activation='softsign', kernel_constraint=maxnorm(4)))
    model.add(Dropout(0.0))
    model.add(Dense(32, kernel_initializer='glorot_uniform', activation='sigmoid'))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [19]:
from tensorflow.keras import regularizers

logdir = os.path.join("logs", "EarlyStopping+L2_WeightDecay")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

model = tf.keras.Sequential([
    Flatten(input_shape=(28,28)),
    Dense(512, kernel_regularizer=regularizers.l2(0.01)),
    ReLU(negative_slope=.01),
    Dense(512, kernel_regularizer=regularizers.l2(0.01)),
    ReLU(negative_slope=.01),
    Dense(512, kernel_regularizer=regularizers.l2(0.01)),
    ReLU(negative_slope=.01),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/99
2500/2500 [==============================] - 38s 15ms/step - loss: 8.3228 - accuracy: 0.7691 - val_loss: 2.2876 - val_accuracy: 0.8186
Epoch 2/99
2500/2500 [==============================] - 37s 15ms/step - loss: 1.2753 - accuracy: 0.8157 - val_loss: 0.9120 - val_accuracy: 0.8179
Epoch 3/99
2500/2500 [==============================] - 37s 15ms/step - loss: 0.8623 - accuracy: 0.8162 - val_loss: 0.8364 - val_accuracy: 0.8193
Epoch 4/99
2500/2500 [==============================] - 37s 15ms/step - loss: 0.8230 - accuracy: 0.8202 - val_loss: 0.9212 - val_accuracy: 0.7965
Epoch 5/99
2500/2500 [==============================] - 37s 15ms/step - loss: 0.8360 - accuracy: 0.8237 - val_loss: 0.8548 - val_accuracy: 0.8138
Epoch 6/99
2500/2500 [==============================] - 38s 15ms/step - loss: 0.8188 - accuracy: 0.8262 - val_loss: 0.9047 - val_accuracy: 0.7947 - loss: 0.8189 - ac


In [20]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
re_lu_15 (ReLU)              (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               262656    
_________________________________________________________________
re_lu_16 (ReLU)              (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 512)               262656    
_________________________________________________________________
re_lu_17 (ReLU)              (None, 512)              

In [17]:
# Your Code Starts Here
logdir = os.path.join("logs", "EarlyStopping-Loss")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

def final_model(neurons=64):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=784, kernel_initializer='glorot_uniform', activation='softsign', kernel_constraint=maxnorm(4)))
    model.add(Dropout(0.0))
    model.add(Dense(32, kernel_initializer='glorot_uniform', activation='sigmoid'))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/99
2500/2500 [==============================] - 4s 2ms/step - loss: 4.1589 - accuracy: 0.0064 - val_loss: 4.1589 - val_accuracy: 0.0073
Epoch 2/99
2500/2500 [==============================] - 4s 2ms/step - loss: 4.1589 - accuracy: 0.0064 - val_loss: 4.1589 - val_accuracy: 0.0073
Epoch 3/99
2500/2500 [==============================] - 4s 2ms/step - loss: 4.1589 - accuracy: 0.0064 - val_loss: 4.1589 - val_accuracy: 0.0073
Epoch 4/99
2500/2500 [==============================] - 4s 2ms/step - loss: 4.1589 - accuracy: 0.0064 - val_loss: 4.1589 - val_accuracy: 0.0073


In [18]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               100480    
_________________________________________________________________
re_lu_12 (ReLU)              (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               16512     
_________________________________________________________________
re_lu_13 (ReLU)              (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               16512     
_________________________________________________________________
re_lu_14 (ReLU)              (None, 128)              

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [79]:
cpoint = tf.keras.callbacks.ModelCheckpoint("weights_best.h6",
                                            verbose=1, 
                                            save_weights_only=True)

def create_model():
  model = tf.keras.Sequential([
      Flatten(input_shape=(1, 784)),
      Dense(128),
      ReLU(negative_slope=.01),
      Dense(128),
      ReLU(negative_slope=.01),
      Dense(128),
      ReLU(negative_slope=.01),
      Dense(10, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
                metrics=['accuracy'])

  return model

model = create_model()

model.fit(X_train, y_train, epochs=32,batch_size=32,
          validation_data=(X_test,y_test),
          verbose=2,
          callbacks=[cpoint])

Epoch 1/32

Epoch 00001: saving model to weights_best.h6
2500/2500 - 4s - loss: 1.5288 - accuracy: 0.6976 - val_loss: 0.7606 - val_accuracy: 0.7771
Epoch 2/32

Epoch 00002: saving model to weights_best.h6
2500/2500 - 4s - loss: 0.6685 - accuracy: 0.7968 - val_loss: 0.6624 - val_accuracy: 0.8041
Epoch 3/32

Epoch 00003: saving model to weights_best.h6
2500/2500 - 4s - loss: 0.5806 - accuracy: 0.8234 - val_loss: 0.6033 - val_accuracy: 0.8196
Epoch 4/32

Epoch 00004: saving model to weights_best.h6
2500/2500 - 4s - loss: 0.5309 - accuracy: 0.8392 - val_loss: 0.5638 - val_accuracy: 0.8347
Epoch 5/32

Epoch 00005: saving model to weights_best.h6
2500/2500 - 4s - loss: 0.4961 - accuracy: 0.8510 - val_loss: 0.5382 - val_accuracy: 0.8423
Epoch 6/32

Epoch 00006: saving model to weights_best.h6
2500/2500 - 4s - loss: 0.4649 - accuracy: 0.8599 - val_loss: 0.5513 - val_accuracy: 0.8409
Epoch 7/32

Epoch 00007: saving model to weights_best.h6
2500/2500 - 4s - loss: 0.4424 - accuracy: 0.8673 - val_

In [80]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 719us/step - loss: 1.0109 - accuracy: 0.8467


[1.0109076499938965, 0.8467000126838684]

In [82]:
# Your Code Starts Here
m = create_model()  # Start with same architecture
m.load_weights('./weights_best.h6')  # Load instead of train
m.summary()

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_43 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_211 (Dense)            (None, 128)               100480    
_________________________________________________________________
re_lu_124 (ReLU)             (None, 128)               0         
_________________________________________________________________
dense_212 (Dense)            (None, 128)               16512     
_________________________________________________________________
re_lu_125 (ReLU)             (None, 128)               0         
_________________________________________________________________
dense_213 (Dense)            (None, 128)               16512     
_________________________________________________________________
re_lu_126 (ReLU)             (None, 128)             

In [83]:
m.evaluate(X_test, y_test)

313/313 [==============================] - 0s 722us/step - loss: 1.0109 - accuracy: 0.8467


[1.0109076499938965, 0.8467000126838684]

In [26]:
def final_model(neurons=64):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=784, kernel_initializer='glorot_uniform', activation='softsign', kernel_constraint=maxnorm(4)))
    model.add(Dropout(0.0))
    model.add(Dense(32, kernel_initializer='glorot_uniform', activation='sigmoid'))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [86]:
cpoint = tf.keras.callbacks.ModelCheckpoint("weights_best.h7",
                                            verbose=1, 
                                            save_weights_only=True)

def create_model():
  model = tf.keras.Sequential([
      Flatten(input_shape=(1, 784)),
      Dense(128),
      ReLU(negative_slope=.01),
      Dense(128,kernel_initializer='glorot_uniform', activation='softsign', kernel_constraint=maxnorm(4)),
      ReLU(negative_slope=.01),
      Dense(128,kernel_initializer='glorot_uniform', activation='sigmoid'),
      ReLU(negative_slope=.01),
      Dense(10, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy', optimizer='Adamax',
                metrics=['accuracy'])

  return model

model = create_model()

model.fit(X_train, y_train, epochs=32,batch_size=32,
          validation_data=(X_test,y_test),
          verbose=2,
          callbacks=[cpoint])

Epoch 1/32

Epoch 00001: saving model to weights_best.h7
2500/2500 - 3s - loss: 1.0959 - accuracy: 0.6631 - val_loss: 0.8528 - val_accuracy: 0.7387
Epoch 2/32

Epoch 00002: saving model to weights_best.h7
2500/2500 - 3s - loss: 0.7883 - accuracy: 0.7558 - val_loss: 0.7728 - val_accuracy: 0.7640
Epoch 3/32

Epoch 00003: saving model to weights_best.h7
2500/2500 - 3s - loss: 0.7286 - accuracy: 0.7743 - val_loss: 0.7392 - val_accuracy: 0.7751
Epoch 4/32

Epoch 00004: saving model to weights_best.h7
2500/2500 - 3s - loss: 0.6938 - accuracy: 0.7839 - val_loss: 0.7084 - val_accuracy: 0.7837
Epoch 5/32

Epoch 00005: saving model to weights_best.h7
2500/2500 - 3s - loss: 0.6666 - accuracy: 0.7943 - val_loss: 0.6936 - val_accuracy: 0.7938
Epoch 6/32

Epoch 00006: saving model to weights_best.h7
2500/2500 - 3s - loss: 0.6379 - accuracy: 0.8024 - val_loss: 0.6690 - val_accuracy: 0.8021
Epoch 7/32

Epoch 00007: saving model to weights_best.h7
2500/2500 - 3s - loss: 0.6249 - accuracy: 0.8056 - val_

In [87]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 697us/step - loss: 0.5478 - accuracy: 0.8349


[0.5477806925773621, 0.8349000215530396]

In [88]:
m = create_model()  # Start with same architecture
m.load_weights('./weights_best.h7')  # Load instead of train
m.summary()

Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_47 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_227 (Dense)            (None, 128)               100480    
_________________________________________________________________
re_lu_136 (ReLU)             (None, 128)               0         
_________________________________________________________________
dense_228 (Dense)            (None, 128)               16512     
_________________________________________________________________
re_lu_137 (ReLU)             (None, 128)               0         
_________________________________________________________________
dense_229 (Dense)            (None, 128)               16512     
_________________________________________________________________
re_lu_138 (ReLU)             (None, 128)             

In [89]:
m.evaluate(X_test, y_test)

313/313 [==============================] - 0s 709us/step - loss: 0.5478 - accuracy: 0.8349


[0.5477806925773621, 0.8349000215530396]

### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 